# Project I: Effective field theory modeling (Bayesian parameter estimation)
## Learning from data [TIF285], Chalmers, Fall 2024

Last revised: 14-Aug-2024 by Christian Forsn [christian.forssen@chalmers.se]

## Instructions

- See deadline on the course web page
- This project is performed in groups of two students. 
- The extra task of the project is optional. See examination rules on the course web page.
- Hand-in your written report and your solution source code via Canvas.
- Students are allowed to discuss together and help each other when solving the problems and working on projects. However, every student must understand their submitted solution in the sense that they should be able to explain and discuss them with a peer or with a teacher.

### Learning goals:
* Apply and extend the Bayesian parameter estimation ideas and techniques from the course in a realistic scientific setting.
* Learn about some diagnostics for Bayesian parameter estimation.
* Perform MCMC sampling on a small-dimensional problem and interpret the results.
* Explore the impact of: *informative* prior knowledge, dependence on data precision and correlations; modeling discrepancy.
* This a less-guided set of tasks and you will have to put together ideas and tools we've discussed.

### Written report
- Page limit: 6 pages (excluding title page and list of references). 3 extra pages are allowed to present the optional extra task.
- Give a short description of the nature of the problem and the methods you have used.
- Include your results as figures and/or tables. All tables and figures should have relevant captions and labels on the axes.
- Try to give an interpretation of you results.
- Upload the source code of your program as a separate file (.ipynb or .py). Commented code might help the grader.

## Main task
The overall project goal is to reproduce various results in a paper: [*Rigorous constraints on three-nucleon forces in chiral effective field theory from fast and accurate calculations of few-body observables*](https://arxiv.org/abs/2104.04441).  It's a long, scientific paper so don't try to read all of it!  Sec. II of the paper is the most relevant, but you don't need to understand all of it. We'll guide you to the important parts. 

**Note**: Don't expect to exactly reproduce the results in the paper. We will be using a slightly different model, and make some additional approximations, compared to the study by Wesolowski et al. Differences will be larger for the values of inferred parameters than they will be for physical observables.

### General modeling and project description
In this project you are faced with a very general scenario for scientific modeling. You have a physics model with a number of unknown parameters. The model outputs can be compared with observed data. Your task is to infer the model parameters given a set of experimental data while taking your knowledge about sources of uncertainty into account. After checking the consistency of the analysis, the calibrated model can then be used for predictions.

The model is implemented on a computer and is costly to evaluate. In a more general scientific setting, this computer implemented model is known as a **simulator**. The computational complexity can sometimes be reduced by model-order reduction. The general idea is to construct an **emulator** that mimics the output of the simulator. The emulator will usually have smaller precision but can be evaluated at a fraction of the computational cost. Having access to a reduced-order model enables a much richer palette of statistical analyses and computer-aided experiments. 

In this particular project, you are modeling the properties of atomic nuclei starting from chiral effective field theory. Specifically, the model can be used to compute the binding energies ($E$) of $^3$H, $^3$He, $^4$He, the (point-proton) radius ($R_p$) of $^4$He and the comparative beta-decay halflife ($fT$) of $^3$H. You have access to experimental data for these observables, as well as some knowledge of the relevant sources of uncertainty. 

The unknown parameters of the model correspond to certain terms in the effective-field theory description of the strong force. They are labeled $c_D$ and $c_E$. Your task is to perform Bayesian parameter estimation and model checking. Important ingredients of this analysis will be to assess the information content of the different data and the incorporation of both experimental uncertainties and theoretical ones. For the extra task you will explore the main theoretical uncertainty, the so called modeling discrepancy term, in more detail.

### Physics background
(This part with the subsections on chiral EFT, many-body theory and eigenvector continuation, is supplied to provide the physics motivation and background for the study. You don't need to include any of this background context in your project report.)

The modeling of hadronic systems, such as atomic nuclei, involves a theoretical description of the strong interaction. At a very fundamental level, such a description involves quarks and gluons within a type of quantum field theory known as quantum chromodynamics (QCD). This is included in the standard model of particle physics. However, at low energies, quarks condense into hadrons and the fully resolved QCD description becomes intractable. An effective theory can be constructed in which neutrons, protons (and possibly other hadrons) are used as effective degrees of freedom. The most general interaction between these particles, consistent with the symmetries of the underlying theory, can be formulated using the framework of effective field theory (EFT). In this project we are using a physics model based on a so called chiral EFT, which utilizes the spontaneously broken chiral symmetry of QCD. 

Important, general aspects of EFTs are:
- An infinite number of interaction diagrams can be generated. However, they can be ordered by importance using a so called power counting. This gives an EFT expansion for which $Q$, the expansion parameter, is an important property as it sets the convergence rate.
- Many diagrams are associated with unknown parameters, so called low-energy constants (LECs), that must be inferred.
- Predictions can be made at a certain truncation of the EFT expansion. Such model predictions will be associated with a truncation error. This modeling uncertainty can be quantified using our knowledge of the EFT expansion.

#### Chiral EFT for nuclei
Atomic nuclei can be modeled as strongly interacting, quantum many-body systems. Using a non-relativistic approximation, the truncated chiral EFT description gives an interaction potential
$$
V^k(\vec\theta) = V^k_{N\!N}(\vec\theta_{N\!N}) + V^k_{3N}(\vec\theta_{3N}) + \ldots,
$$
where $k$ is the chiral expansion order, $\vec\theta$ is the vector of LECs (model parameters), and the unresolved short-distance physics leads to the appearance of irreducible higher-body forces. In this particular case, we will be considering a scenario where we are operating at $k=3$, known as next-to-next-to-leading order (NNLO), at which three-nucleon forces appear for the first time. We will also make the simplifying assumption that LECs associated with two-nucleon diagrams have already been inferred to infinite precision. This implies that the parameters $\vec\theta_{N\!N}$ are known, but that $\vec\theta_{3N}$ remains to be determined. The parameter vector $\vec\theta_{3N}$ consists of two LECs that are known as $(c_D, c_E)$. The analysis performed in Wesolowski's paper is more involved as it includes a prior uncertainty in $\vec\theta_{N\!N}$. This will not be considered here.

#### Solving the nuclear many-body problem 
We are interested in solving the non-relativistic Schrödinger equation $H \Psi = E \Psi$ for a many-body system. The Hamiltonian will contain the chiral EFT potential plus kinetic energy terms. The eigenvalue $E$ corresponds to the total energy of the system, whereas expectation values of other operators give further observables such as radii and transition strengths.

The solution of the many-body Schrödinger equation can be approached by constructing a single-particle basis and using the formalism of second quantization to obtain a basis for the many-body system under consideration. Projecting the Hamiltonian operator on this (high-dimensional) basis leads to a large-scale eigenvalue problem that can be solved iteratively. For the few-nucleon systems considered here the Hamiltonian matrix can be of size $N \times N$ with $N \approx 10^5$. 

#### Eigenvector continuation emulators
Using an eigensolver developed at Chalmers, the lowest eigenvalues and corresponding eigenstates of $A=3,4$ systems can be found using Lanczos diagonalization in $\approx 60$ seconds. While this is good, it is still not enough for a statistical analysis in which you would like to change parameter values and solve the problem repeatedly. In this project you will be using a reduced-order model that mimics the full solution in just a fraction of the time (milliseconds rather than minutes). It relies on a technique known as eigenvector continuation (EC) that finds an optimal basis and translates the large-scale ($N \approx 10^5$) eigenvalue problem into a small-scale ($N_{EC} \approx 10$) generalized eigenvalue problem. The emulators are described in Sec. III.B of the paper with some relevant references for the interested reader. Again, there is no need to include any of this in your report.

## Tasks

### Task 1: Studies of the posterior with individual data likelihood
- Study the importance of the four observables (E4He, Rp4He, E3H, fT3H) listed in Table I of the paper by computing the four different posteriors
$$
p(c_D, c_E | y_{\mathrm{exp},i}, \sigma_i, I ) \propto p(y_{\mathrm{exp},i} | c_D, c_E, \sigma_i, I) p(c_D, c_E | I ),
$$
  that result from inference given a single datum $y_{\mathrm{exp},i} \in \{\text{E4He, Rp4He, E3H, fT3H}\}$ from the above list and the associated error model represented by $\sigma_i, I$.
- This requires to compute the individual data likelihood for each observable separately as a function of the model parameters $(c_D, c_E) \equiv \vec\theta_{3N}$.
- More specifically, you should try to reproduce figures 6a and 6b from the paper. Remember that you will not get exactly the same results as in the paper due to slight differences in the underlying model and approximations. Specifically, **the MAP point will be different** compared with the paper.
- The parameters $c_D, c_E$ can be positive or negative, but are expected to be of natural size. We encode this expectation into the prior
$$
p(c_D, c_E | I) = \mathcal{N}(c_D; \mu=0, \sigma=5) \mathcal{N}(c_E; \mu=0, \sigma=5).
$$
  That is independent, normal distributions with mean zero and standard deviation $\sigma=5$.
- You can assume that each likelihood is given by a normal distribution
$$
p(y_{\mathrm{exp},i} | c_D, c_E, \sigma_i, I) = \mathcal{N} \left( y_{\mathrm{th},i}(c_D, c_E), \sigma_i \right).
$$
  I.e., the mean of the respective distribution is the model prediction $y_{\mathrm{th},i}(c_D, c_E)$ for that observable ($i$) for the given set of model parameters, while the standard deviation $\sigma_i$ reflects the combined experimental and theoretical uncertainties for that observable. This likelihood will be evaluated at the point corresponding to the observed, experimental value $y_{\mathrm{exp},i}$.
- To reproduce Fig. 6a you should use the adopted errors that are presented in Table I. They quantify the combination of experimental error and estimated precision of the many-body solver. That is, you should use $\sigma_i = \sigma_{\mathrm{exp+method},i}$
- To reproduce Fig. 6b you should add a model discrepancy term to your error model such that the total variance becomes $\sigma_i^2 = \sigma_{\mathrm{exp+method},i}^2 + \sigma_{\mathrm{EFT},i}^2$. In this case, the second term (known as a model discrepancy term) quantifies the uncertainty that comes from the truncation of the EFT expansion. Following the paper you can estimate this error with the formula
$$
\sigma_{\mathrm{EFT},i}^2 = \frac{ \left( y_{\mathrm{exp},i} \bar{c} Q^{k+1} \right)^2}{1 - Q^2},
$$
  where $k=3$ and you can fix $\bar{c}=1$ and the expansion parameter $Q=0.33$.

#### Suggestions how to proceed and what to consider
- Use the function `fewnucleonEmulator` in the supplied python module `quantumsolver` (see below) to compute the range of relevant few-nucleon observables for different model input (parameters $c_D, c_E$). Read the docstring of `fewnucleonEmulator` to understand the input arguments and the output. It is suggested to do this on a grid $c_D \in [-2.5, 2.5]$, $c_E \in [-1.0, 1.0]$
- How dense grid can you use given that each model evaluation takes $\lesssim 1$ ms and you would probably not like to wait more than $\sim$ minutes?
- You will probably find that grid evaluation is very inefficient for reproducing these narrow PDFs. You are allowed to plot another iso-probability contour if you find that 39% region showed in the paper is too tight. 
- There might be useful utility code in Problem Set 2 for extracting contour levels for a PDF that is evaluated on a grid.
- How many data points would you naively need to infer two model parameters? If you were to combine the data that you have available, which set do you think works best for this inference problem?

In [1]:
import numpy as np
# add additional import statements as needed

Note that `quantumsolver.py` and the data directory `evcData` must be in the same directory as the notebook where the following import statement is evaluated.

In [8]:
import quantumsolver as qs

The relevant function is `fewnucleonEmulator`. You can run

```
help(qs.fewnucleonEmulator)
```

to read the function docstring.

As a test that everything works you can run

```
qs.fewnucleonEmulator(qs.cD_NNLOsat,qs.cE_NNLOsat)
```

which should produce the output: (-28.423318811617758, 1.4824198390733887, 1.6987274146484983, -7.75962047220291, -8.517539812967788, 0.6794354970057697, 1144.0855457148618)

[Credit to Andreas Ekström (Dept. of Physics, Chalmers) for producing the EVC emulator that is used in `fewnucleonEmulator`.]

To avoid for loops when evaluatiung the model on a grid, you might want to create a vectorized version. This can be done with `np.vectorize`. See below including an illustration how it works.

In [9]:
# Vectorized version of the fewnucleonEmulator
vfewnucleonEmulator = np.vectorize(qs.fewnucleonEmulator)

In [12]:
# Test of vectorized version
cD_grid = np.array([-2.5, 2.5])
cE_grid = np.array([-1., 1.])
# Create a 2-dimensional grid
cD_eval, cE_eval = np.meshgrid(cD_grid,cE_grid)
print('cD_eval is of shape:', cD_eval.shape)
print(cD_eval)
# The vectorized version of the few-nucleon emulator will produce an output tuple
# where each element is an array of the same shape as the input
E4He_eval, Rp4He_eval, Rch4He_eval, E3He_eval, E3H_eval, E1A3H_eval, fT3H_eval = vfewnucleonEmulator(cD_eval, cE_eval)
print('E4He_eval is of shape:', E4He_eval.shape)
print(E4He_eval)

cD_eval is of shape: (2, 2)
[[-2.5  2.5]
 [-2.5  2.5]]
E4He_eval is of shape: (2, 2)
[[ -24.40247682  -21.37114724]
 [-105.55765698  -40.33178611]]


## Task 2: Sample the posterior
- Perform the inference using MCMC sampling (for example using the `emcee` package) and two different choices for the combined data likelihood that includes two (or four) observables:
  - Case 1: $\{E(^4\mathrm{He}), R_p(^4\mathrm{He})\}$
  - Case 2: $\{E(^4\mathrm{He}), R_p(^4\mathrm{He}), E(^3\mathrm{H}), fT(^3\mathrm{H})\}$
- Include the fixed EFT errors that were computed in the previous task.
- You can assume that all errors are independent, which implies that the total likelihood is just a product of the individual ones.
- Represent the results in a corner plot. I.e., you will basically reproduce the result in Fig. 3 in the paper (which is conditioned on all four observables, i.e., Case 2). Your parameter distribution will be somewhat different partly due to the fixed model discrepancy.
- Check the convergence of your MCMC sampling (at least visually by plotting the traces). 

#### Comments and suggestions
* Define functions for the `log_prior`, the `log_likelihood` and the `log_posterior`. The latter is provided as input to the `emcee` sampler. Note that the likelihood (and therefore the posterior) will depend on the choice of data set.
* Operating with log probabilities, the product of independent ones turns into a simple sum.
* `scipy.stats` provides a useful library for working with continuous variable distributions and for evaluating `logpdf`.
* It is probably worthwhile to test your `log_posterior` function before starting the MCMC sampling. 
* Consider beforehand how many samples you can afford to collect.
* The `show_titles=True` option to `corner` will show central results and one-$\sigma$ error limits on the projected posterior plots.
* The `quantiles=[0.16, 0.5, 0.84]`option to corner adds the dashed vertical lines to the marginal posteriors on the diagonal. You can obviously change the quantiles if you want another credibility region.
* The python command `np.percentile(y, [16, 50, 84], axis=0)` might be useful to extract numerical values for the credibility region and the median from a python array `y`.

## Task 3: Posterior predictive distributions
- Use samples from the posterior distribution obtained with the fulll data set (Case 2 of Task 2) and produce a posterior predictive distribution for the four observables. That is, you should reproduce a result similar to the one shown in Fig. 4.
- Represent the results in a corner plot. I.e., you will basically reproduce the result in Fig. 4 in the paper. Your results will be somewhat different partly due to the fixed model discrepancy.
- As the predicted observables are the same ones that were used for calibrating the model (i.e., entering the likelihood), this should be considered as a form of model checking. We are testing that the model is consistent and that the data analysis worked.

## Extra task: Sample the error model

Assume that we would like to learn more about our error model, in particular for the model discrepancy term. Can we incorporate an uncertainty about this error in the analysis? Yes, within the bayesian framework this is relatively straightforward. For example, the expansion parameter $Q$ that enters in the variance of the model discrepancy term can be considered a model parameter. In the paper by Wesolowski et al, also the scale $\bar{c}$ was inferred but we will keep it fixed as before.

- The task is now to infer the joint posterior for $c_D, c_E, Q$ given the data and the error model ($\sigma_\mathrm{EFT}$ will not be fixed). 
- Concerning prior information for $Q$ we can use the arguments listed on page 4 in the paper and assign a weakly informative Beta distribution (with parameters $a=3$, $b=5$) for its prior. That is $p(Q|I) = B(Q | a = 3, b = 5)$.
- Try to reproduce Figs. 3 and 4 in the paper. Apart from some differences in the model parameters, the distributions should look more similar than they did in tasks 2 and 3.
- You can also glance at Fig. 5, although you are actually not varying $\bar{c}$.

#### Suggestions how to proceed and what to consider
- You might need more samples or more walkers for this case
- Are there visible differences in the shape of the extracted distributions? Why?
- The keyword argument `range` can be used in the `corner` package to set the fraction of samples that will be shown. It is useful if there are some outliers that would force the plot range to be very large.
- Are we learning anything about $Q$ from this analysis?